# This is a Kaggle competition to predict housing prices
## I will be comparing which model to use: XGB, Random Forest or MLR


Lets first load required libraries:

In [311]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import pyodbc
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers
from keras.models  import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

Lets download the dataset

In [312]:
df1 = pd.read_csv('C:/Users/taihs/OneDrive/Documents/house prices/train.csv')

In [313]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [314]:
co=df1.corr()
co['SalePrice'].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [315]:
df = df1[['OverallQual','GrLivArea','GarageCars','GarageArea','TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','YearBuilt',\
        'YearRemodAdd','GarageYrBlt','MasVnrArea','Fireplaces','BsmtFinSF1','LotFrontage','WoodDeckSF','2ndFlrSF','OpenPorchSF',\
        'HalfBath','LotArea','BsmtFullBath','BsmtUnfSF','BedroomAbvGr']]
df.head()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,BsmtFinSF1,LotFrontage,WoodDeckSF,2ndFlrSF,OpenPorchSF,HalfBath,LotArea,BsmtFullBath,BsmtUnfSF,BedroomAbvGr
0,7,1710,2,548,856,856,2,8,2003,2003,...,706,65.0,0,854,61,1,8450,1,150,3
1,6,1262,2,460,1262,1262,2,6,1976,1976,...,978,80.0,298,0,0,0,9600,0,284,3
2,7,1786,2,608,920,920,2,6,2001,2002,...,486,68.0,0,866,42,1,11250,1,434,3
3,7,1717,3,642,756,961,1,7,1915,1970,...,216,60.0,0,756,35,0,9550,1,540,3
4,8,2198,3,836,1145,1145,2,9,2000,2000,...,655,84.0,192,1053,84,1,14260,1,490,4


In [316]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1460 non-null   int64  
 1   GrLivArea     1460 non-null   int64  
 2   GarageCars    1460 non-null   int64  
 3   GarageArea    1460 non-null   int64  
 4   TotalBsmtSF   1460 non-null   int64  
 5   1stFlrSF      1460 non-null   int64  
 6   FullBath      1460 non-null   int64  
 7   TotRmsAbvGrd  1460 non-null   int64  
 8   YearBuilt     1460 non-null   int64  
 9   YearRemodAdd  1460 non-null   int64  
 10  GarageYrBlt   1379 non-null   float64
 11  MasVnrArea    1452 non-null   float64
 12  Fireplaces    1460 non-null   int64  
 13  BsmtFinSF1    1460 non-null   int64  
 14  LotFrontage   1201 non-null   float64
 15  WoodDeckSF    1460 non-null   int64  
 16  2ndFlrSF      1460 non-null   int64  
 17  OpenPorchSF   1460 non-null   int64  
 18  HalfBath      1460 non-null 

In [317]:
df = df.fillna(0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1460 non-null   int64  
 1   GrLivArea     1460 non-null   int64  
 2   GarageCars    1460 non-null   int64  
 3   GarageArea    1460 non-null   int64  
 4   TotalBsmtSF   1460 non-null   int64  
 5   1stFlrSF      1460 non-null   int64  
 6   FullBath      1460 non-null   int64  
 7   TotRmsAbvGrd  1460 non-null   int64  
 8   YearBuilt     1460 non-null   int64  
 9   YearRemodAdd  1460 non-null   int64  
 10  GarageYrBlt   1460 non-null   float64
 11  MasVnrArea    1460 non-null   float64
 12  Fireplaces    1460 non-null   int64  
 13  BsmtFinSF1    1460 non-null   int64  
 14  LotFrontage   1460 non-null   float64
 15  WoodDeckSF    1460 non-null   int64  
 16  2ndFlrSF      1460 non-null   int64  
 17  OpenPorchSF   1460 non-null   int64  
 18  HalfBath      1460 non-null 

In [318]:
X = df
y = df1['SalePrice']

In [191]:
X= preprocessing.StandardScaler().fit(X).transform(X)

In [319]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

### XGB

In [320]:
xgb = XGBRegressor(n_estimators=200)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [321]:
y_predict = xgb.predict(X_test)

In [322]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math

r2 = r2_score(y_test,y_predict)
mse = mean_squared_error(y_test,y_predict)
rmse = math.sqrt(mse)

print('r2',r2)
print('rmse',rmse)

r2 0.8655022954817586
rmse 27326.416394042895


### RandomForest

In [323]:
rf = RandomForestRegressor(n_estimators=200)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200)

In [324]:
y_predict=rf.predict(X_test)

r2 = r2_score(y_test,y_predict)
mse = mean_squared_error(y_test,y_predict)
rmse = math.sqrt(mse)

print('r2',r2)
print('rmse',rmse)

r2 0.8855691267802577
rmse 25205.58907024397


### Polynomial

In [325]:
from sklearn import linear_model

clf = linear_model.LinearRegression()
clf.fit(X_train, y_train)

LinearRegression()

In [326]:
y_predict = clf.predict(X_test)

r2 = r2_score(y_test,y_predict)
mse = mean_squared_error(y_test,y_predict)
rmse = math.sqrt(mse)

print('r2',r2)
print('rmse',rmse)

r2 0.8418742142763243
rmse 29629.654943152094


### Decision Tree

In [327]:
tree = DecisionTreeRegressor(max_depth = 8)

In [328]:
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=8)

In [329]:
y_predict = tree.predict(X_test)

r2 = r2_score(y_test,y_predict)
mse = mean_squared_error(y_test,y_predict)
rmse = math.sqrt(mse)

print('r2',r2)
print('rmse',rmse)

r2 0.7727373100740127
rmse 35521.33806405278


### Load the test data

In [330]:
df1 = pd.read_csv('C:/Users/taihs/OneDrive/Documents/house prices/test.csv')

In [331]:
df = df1[['OverallQual','GrLivArea','GarageCars','GarageArea','TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','YearBuilt',\
        'YearRemodAdd','GarageYrBlt','MasVnrArea','Fireplaces','BsmtFinSF1','LotFrontage','WoodDeckSF','2ndFlrSF','OpenPorchSF',\
        'HalfBath','LotArea','BsmtFullBath','BsmtUnfSF','BedroomAbvGr']]
df.head()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,BsmtFinSF1,LotFrontage,WoodDeckSF,2ndFlrSF,OpenPorchSF,HalfBath,LotArea,BsmtFullBath,BsmtUnfSF,BedroomAbvGr
0,5,896,1.0,730.0,882.0,896,1,5,1961,1961,...,468.0,80.0,140,0,0,0,11622,0.0,270.0,2
1,6,1329,1.0,312.0,1329.0,1329,1,6,1958,1958,...,923.0,81.0,393,0,36,1,14267,0.0,406.0,3
2,5,1629,2.0,482.0,928.0,928,2,6,1997,1998,...,791.0,74.0,212,701,34,1,13830,0.0,137.0,3
3,6,1604,2.0,470.0,926.0,926,2,7,1998,1998,...,602.0,78.0,360,678,36,1,9978,0.0,324.0,3
4,8,1280,2.0,506.0,1280.0,1280,2,5,1992,1992,...,263.0,43.0,0,0,82,0,5005,0.0,1017.0,2


In [332]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1459 non-null   int64  
 1   GrLivArea     1459 non-null   int64  
 2   GarageCars    1458 non-null   float64
 3   GarageArea    1458 non-null   float64
 4   TotalBsmtSF   1458 non-null   float64
 5   1stFlrSF      1459 non-null   int64  
 6   FullBath      1459 non-null   int64  
 7   TotRmsAbvGrd  1459 non-null   int64  
 8   YearBuilt     1459 non-null   int64  
 9   YearRemodAdd  1459 non-null   int64  
 10  GarageYrBlt   1381 non-null   float64
 11  MasVnrArea    1444 non-null   float64
 12  Fireplaces    1459 non-null   int64  
 13  BsmtFinSF1    1458 non-null   float64
 14  LotFrontage   1232 non-null   float64
 15  WoodDeckSF    1459 non-null   int64  
 16  2ndFlrSF      1459 non-null   int64  
 17  OpenPorchSF   1459 non-null   int64  
 18  HalfBath      1459 non-null 

In [333]:
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1459 non-null   int64  
 1   GrLivArea     1459 non-null   int64  
 2   GarageCars    1459 non-null   float64
 3   GarageArea    1459 non-null   float64
 4   TotalBsmtSF   1459 non-null   float64
 5   1stFlrSF      1459 non-null   int64  
 6   FullBath      1459 non-null   int64  
 7   TotRmsAbvGrd  1459 non-null   int64  
 8   YearBuilt     1459 non-null   int64  
 9   YearRemodAdd  1459 non-null   int64  
 10  GarageYrBlt   1459 non-null   float64
 11  MasVnrArea    1459 non-null   float64
 12  Fireplaces    1459 non-null   int64  
 13  BsmtFinSF1    1459 non-null   float64
 14  LotFrontage   1459 non-null   float64
 15  WoodDeckSF    1459 non-null   int64  
 16  2ndFlrSF      1459 non-null   int64  
 17  OpenPorchSF   1459 non-null   int64  
 18  HalfBath      1459 non-null 

In [334]:
y_predict_xgb = xgb.predict(df)
y_predict_rf = rf.predict(df)
y_predict_clf = clf.predict(df)
y_predict_tree = tree.predict(df)

In [335]:
final = pd.DataFrame({'SalePrice':y_predict_rf})

In [336]:
final1 = pd.concat([df1['Id'], final], axis=1)
final1.head()

,Id,SalePrice
0,1461,127412.580
1,1462,151989.340
2,1463,185512.650
3,1464,187559.250
4,1465,194319.805


In [337]:
final1.to_csv('sample_submission.csv', index=False)